In [1]:
import numpy as np
import pandas as pd
from calendar import isleap
import glob

In [2]:
# function to find lines to skip in weather files
def find_line(path, target):
    with open(path) as fh:
        count = 1
        for line in fh:
#             print(line.strip()[0:5])
            if line.strip()[0:5] == target:
                return count
            count += 1
    return 0

In [3]:
# 47 station coordinates
coords = pd.read_csv("coords1.csv")
stn = coords["stn"]
lon = coords["lon"]
lat = coords["lat"]
# lat

In [26]:
all_stn_wth_files_tmax = pd.DataFrame(columns=["DATE"])

for station in stn:
    print(station)
    all_wth_files_tmax = []
    # read each station data from weather files
    weather_files = glob.glob("Weather/Weather/" + station + "*.WTH")
    for file in weather_files:
#         print(weather_files)
        to_skip = find_line(path=file, target="@DATE")
        wth_file = pd.read_fwf(file, skiprows = to_skip, 
                               usecols = [0,2,3,4],
                               names = ["DATE", "TMAX", "TMIN", "RAIN"],
                               dtype={"DATE":str})
        wth_file.DATE[0][0:2]
        if float(wth_file.DATE[0][0:2]) < 50:
            year = "20" + wth_file.DATE[0][0:2]
        else:
            year = "19" + wth_file.DATE[0][0:2]
#             print(year)
        wth_file["YEAR"] = year
        wth_file["DOY"] = list(range(1,len(wth_file)+1))
        
        if isleap(float(year)):
            days_in_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        else:
            days_in_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
            
        print(file, year, to_skip, wth_file.DATE[0][0:2], len(wth_file.columns), days_in_month)
            
        wth_file["DAYS"] = np.concatenate([list(range(1, i+1)) for i in days_in_month])
        wth_file["MONTHS"] = np.concatenate([[j]*i for i, j in zip(days_in_month, list(range(1,13)))])
        wth_file["DATE"] = wth_file["DAYS"].astype(str) + "/" + wth_file["MONTHS"].astype(str) + "/" + wth_file["YEAR"]
        wth_file_tmax = wth_file[["DATE","TMAX"]]
        wth_file_tmax.columns = ["DATE", station]
        all_wth_files_tmax.append(wth_file_tmax)
    all_wth_files_tmax = pd.concat(all_wth_files_tmax, axis = 0)
    
    all_stn_wth_files_tmax = all_stn_wth_files_tmax.merge(all_wth_files_tmax, left_on="DATE", right_on="DATE", how = "outer")




MAID
Weather/Weather\MAID0001.WTH 2000 6 00 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\MAID0101.WTH 2001 6 01 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\MAID0201.WTH 2002 6 02 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\MAID0301.WTH 2003 6 03 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\MAID0401.WTH 2004 6 04 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\MAID0501.WTH 2005 6 05 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\MAID0601.WTH 2006 6 06 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\MAID0701.WTH 2007 6 07 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\MAID0801.WTH 2008 6 08 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\MAID0901.WTH 2009 6 09 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\MAID1001.WTH 2010 6 10 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weath

Weather/Weather\ABEO0001.WTH 2000 7 00 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\ABEO0101.WTH 2001 7 01 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\ABEO0201.WTH 2002 7 02 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\ABEO0301.WTH 2003 7 03 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\ABEO0401.WTH 2004 7 04 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\ABEO0501.WTH 2005 7 05 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\ABEO0601.WTH 2006 7 06 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\ABEO0701.WTH 2007 7 07 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\ABEO0801.WTH 2008 7 08 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\ABEO0901.WTH 2009 7 09 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\ABEO1001.WTH 2010 7 10 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/We

Weather/Weather\ADEK9701.WTH 1997 6 97 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\ADEK9801.WTH 1998 6 98 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\ADEK9901.WTH 1999 6 99 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
AKUR
Weather/Weather\AKUR0001.WTH 2000 6 00 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AKUR0101.WTH 2001 6 01 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AKUR0201.WTH 2002 6 02 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AKUR0301.WTH 2003 6 03 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AKUR0401.WTH 2004 6 04 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AKUR0501.WTH 2005 6 05 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AKUR0601.WTH 2006 6 06 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AKUR0701.WTH 2007 6 07 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weath

AWKA
Weather/Weather\AWKA0001.WTH 2000 6 00 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AWKA0101.WTH 2001 6 01 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AWKA0201.WTH 2002 6 02 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AWKA0301.WTH 2003 6 03 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AWKA0401.WTH 2004 6 04 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AWKA0501.WTH 2005 6 05 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AWKA0601.WTH 2006 6 06 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AWKA0701.WTH 2007 6 07 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AWKA0801.WTH 2008 6 08 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AWKA0901.WTH 2009 6 09 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\AWKA1001.WTH 2010 6 10 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weath

Weather/Weather\BENI1301.WTH 2013 6 13 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\BENI1401.WTH 2014 6 14 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\BENI1501.WTH 2015 6 15 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\BENI1601.WTH 2016 5 16 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\BENI1701.WTH 2017 6 17 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\BENI1801.WTH 2018 6 18 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\BENI1901.WTH 2019 6 19 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\BENI2001.WTH 2020 6 20 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\BENI2101.WTH 2021 6 21 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\BENI7101.WTH 1971 6 71 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\BENI7201.WTH 1972 6 72 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/We

...

Weather/Weather\PORT0301.WTH 2003 6 03 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\PORT0401.WTH 2004 6 04 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\PORT0501.WTH 2005 6 05 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\PORT0601.WTH 2006 6 06 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\PORT0701.WTH 2007 6 07 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\PORT0801.WTH 2008 6 08 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\PORT0901.WTH 2009 6 09 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\PORT1001.WTH 2010 6 10 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\PORT1101.WTH 2011 6 11 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\PORT1201.WTH 2012 6 12 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\PORT1301.WTH 2013 6 13 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/We

SHAK
Weather/Weather\SHAK0001.WTH 2000 6 00 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\SHAK0101.WTH 2001 6 01 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\SHAK0201.WTH 2002 6 02 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\SHAK0301.WTH 2003 6 03 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\SHAK0401.WTH 2004 6 04 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\SHAK0501.WTH 2005 6 05 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\SHAK0601.WTH 2006 6 06 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\SHAK0701.WTH 2007 6 07 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\SHAK0801.WTH 2008 6 08 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\SHAK0901.WTH 2009 6 09 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\SHAK1001.WTH 2010 6 10 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weath

Weather/Weather\UMUA0701.WTH 2007 6 07 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\UMUA0801.WTH 2008 6 08 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\UMUA0901.WTH 2009 6 09 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\UMUA1001.WTH 2010 6 10 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\UMUA1101.WTH 2011 6 11 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\UMUA1201.WTH 2012 6 12 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\UMUA1301.WTH 2013 6 13 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\UMUA1401.WTH 2014 6 14 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\UMUA1501.WTH 2015 6 15 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\UMUA1601.WTH 2016 6 16 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\UMUA1701.WTH 2017 6 17 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/We

Weather/Weather\WARR7301.WTH 1973 6 73 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\WARR7401.WTH 1974 6 74 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\WARR7501.WTH 1975 6 75 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\WARR7601.WTH 1976 6 76 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\WARR7701.WTH 1977 6 77 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\WARR7801.WTH 1978 6 78 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\WARR7901.WTH 1979 6 79 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\WARR8001.WTH 1980 6 80 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\WARR8101.WTH 1981 6 81 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\WARR8201.WTH 1982 6 82 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\WARR8301.WTH 1983 6 83 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/We

Weather/Weather\YOLA1601.WTH 2016 6 16 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\YOLA1701.WTH 2017 6 17 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\YOLA1801.WTH 2018 6 18 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\YOLA1901.WTH 2019 6 19 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\YOLA2001.WTH 2020 6 20 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\YOLA2101.WTH 2021 6 21 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\YOLA7101.WTH 1971 6 71 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\YOLA7201.WTH 1972 6 72 6 [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\YOLA7301.WTH 1973 6 73 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\YOLA7401.WTH 1974 6 74 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/Weather\YOLA7501.WTH 1975 6 75 6 [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
Weather/We

In [48]:
all_stn_wth_files_tmax["DATE"] = pd.to_datetime(all_stn_wth_files_tmax["DATE"])
all_stn_wth_files_tmax = all_stn_wth_files_tmax.sort_values("DATE")
all_stn_wth_files_tmax

,DATE,MAID,MAKU,ABEO,ABUJ,ADEK,AKUR,ASAB,AWKA,BAUC,...,PORT,POTI,SHAK,SOKO,UMUA,UYO0,WARR,YELW,YOLA,ZARI
8036,1971-01-01,33.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,33.0,...,32.0,33.0,NaN,35.0,NaN,NaN,33.0,38.0,36.0,32.0
8067,1971-01-02,33.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,32.0,...,34.0,31.0,NaN,33.0,NaN,NaN,34.0,34.0,36.0,31.0
8095,1971-01-03,37.0,37.0,NaN,NaN,NaN,NaN,NaN,NaN,34.0,...,33.0,35.0,NaN,38.0,NaN,NaN,33.0,36.0,38.0,34.0
8126,1971-01-04,41.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,31.0,...,32.0,38.0,NaN,40.0,NaN,NaN,34.0,32.0,34.0,33.0
8156,1971-01-05,41.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,39.0,...,35.0,41.0,NaN,41.0,NaN,NaN,33.0,39.0,40.0,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8031,2021-12-27,28.2,34.2,36.6,35.7,34.2,34.5,33.1,35.5,29.6,...,32.1,30.0,34.0,31.5,33.8,35.0,36.5,35.0,33.0,28.2
8032,2021-12-28,28.6,35.0,36.5,35.8,35.5,35.0,32.2,36.5,30.2,...,31.2,29.4,34.5,31.8,33.0,35.5,35.7,35.0,33.7,28.6
8033,2021-12-29,29.3,35.2,36.0,35.2,35.0,35.8,31.0,37.0,29.7,...,31.0,29.6,34.0,31.3,33.4,35.5,36.7,36.0,33.3,29.3
8034,2021-12-30,27.8,36.0,36.4,35.0,34.8,34.6,33.4,36.7,30.0,...,34.2,29.1,34.3,31.0,33.8,35.5,36.8,36.0,33.6,27.8


In [ ]:
# wth_file.DATE[0][0:2]
# if float(wth_file.DATE[0][0:2]) < 50:
#     year = "20" + wth_file.DATE[0][0:2]
# else:
#     year = "19" + wth_file.DATE[0][0:2]
# print(year)

# wth_file["YEAR"] = year
# wth_file["DOY"] = list(range(1,len(wth_file)+1))
# wth_file 

In [ ]:
# if isleap(float(year)):
#     days_in_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
# else:
#     days_in_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

# wth_file["DAYS"] = np.concatenate([list(range(1, i+1)) for i in days_in_month])
# wth_file["MONTHS"] = np.concatenate([[j]*i for i, j in zip(days_in_month, list(range(1,13)))])

# wth_file["DATE"] = wth_file["DAYS"].astype(str) + "/" + wth_file["MONTHS"].astype(str) + "/" + wth_file["YEAR"]

# wth_file

# wth_file_tmax = wth_file[["DATE","TMAX"]]
# wth_file_tmax.columns = ["DATE", "STATION"]
# wth_file_tmax
